<a href="https://colab.research.google.com/github/jmcaussade/Analitica_textual_con_ML_PF/blob/main/Proyecto_Final_Dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

In [59]:
!pip install tqdm
!pip install datasets
!pip install langdetect
!pip install num2words

In [60]:
import pandas as pd
import os
import re
import re
from num2words import num2words
from tqdm import tqdm
from datasets import load_dataset
from langdetect import detect
from transformers import GPT2Tokenizer

## Para trabajar con un pedazo del dataset orginal

In [69]:
## SAMPLE DEL DATASET
dataset = load_dataset("somosnlp/recetas-cocina")

In [70]:
df = dataset['train'].to_pandas()

# Sample the first 5000 rows from the DataFrame
#df = df.head(5000)

# Show the first few rows of the sampled data
df.head()

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25


## DATASET COMLETO

In [71]:
df

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho Trifásico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ...",86af61e6-e16a-11ed-bcf5-a96d6180cd25
3,Sudado de Posta o Muchacho,https://www.mycolombianrecipes.com/es/sudado-d...,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ...",86af61e7-e16a-11ed-8914-a96d6180cd25
4,Sopa de Lentejas Colombiana,https://www.mycolombianrecipes.com/es/sopa-de-...,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...,86af71bc-e16a-11ed-b130-a96d6180cd25
...,...,...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,https://www.recetasgratis.net/receta-de-berenj...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...,b0a6454b-a023-44af-a189-92dee8948490
28234,Puré de patatas con queso y huevo,https://www.recetasgratis.net/receta-de-pure-d...,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p...",6e52da68-9496-47de-b85c-7be7fd0b6ac2
28235,Rollitos de calabacín rellenos de jamón y queso,https://www.recetasgratis.net/receta-de-rollit...,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...,22412360-37fe-47de-9401-d3f1e95fe4e7
28236,Calabacín relleno de quinoa,https://www.recetasgratis.net/receta-de-calaba...,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...,eedfabb6-d14f-46e4-a34f-dc07b9015636


In [72]:
df = df.drop(columns=["url", "uuid"])
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [73]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:")
print(missing_values)

Missing Values:
title          5597
ingredients       1
steps             0
dtype: int64


In [74]:
# Drop rows where 'title'  y "ingredients" is null
df = df.dropna(subset=['title'])
df = df.dropna(subset=['ingredients'])


In [75]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3223


In [76]:
# Drop duplicate rows
df = df.drop_duplicates()

In [77]:
print(df.dtypes)

title          object
ingredients    object
steps          object
dtype: object


In [78]:
df

,title,ingredients,steps
0,Arepas de Queso,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maíz, agua caliente, el q..."
1,Sudado de Pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal..."
2,Sancocho Trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"Coloque la cebolla, el pimientón, el ajo y el ..."
3,Sudado de Posta o Muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","En una olla grande ponga la carne, el caldo, ..."
4,Sopa de Lentejas Colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,Cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
28233,Berenjenas rellenas de carne picada y bechamel...,4 Berenjenas 2 Zanahorias 1 Cebolla 1 diente d...,1 Empezaremos a preparar estas berenjenas rell...
28234,Puré de patatas con queso y huevo,2 Patatas 2 Huevos 100 gramos de Parmesano 2 l...,"1 Cocemos las patatas y, una vez listas, las p..."
28235,Rollitos de calabacín rellenos de jamón y queso,½ Calabacín 100 gramos de Jamón cocido 100 gra...,1 Lo primero que debemos hacer es lavar bien e...
28236,Calabacín relleno de quinoa,4 Calabacines 1 Cebolla 1 Pimiento verde 1 Pim...,1 Lo primero que tenemos que hacer para elabor...


In [79]:
df.shape[0]

19417

In [80]:
df['title'] = df['title'].str.lower()

df['ingredients'] = df['ingredients'].str.lower()

# Convert all text in the 'steps' column to lowercase
df['steps'] = df['steps'].str.lower()


,title,ingredients,steps
0,arepas de queso,1 taza de harina de arepa blanca o amarilla\r\...,"combine la harina de maíz, agua caliente, el q..."
1,sudado de pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"en una olla grande, caliente el aceite vegetal..."
2,sancocho trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"coloque la cebolla, el pimientón, el ajo y el ..."
3,sudado de posta o muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","en una olla grande ponga la carne, el caldo, ..."
4,sopa de lentejas colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...
7490,receta fácil y deliciosa,12 discos de empanada 500 gramos de osobuco 1 ...,1 para empezar con la receta de las empanadas ...
7491,receta fácil y saludable,6 tapas de empanadas vegetarianas o veganas 1 ...,1 para empezar con la receta de las empanadas ...
7492,canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 para empezar con la receta de las canastitas...
7493,tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 para empezar con la receta de tacos de canas...


In [ ]:
# Display the updated DataFrame
df.head(1000)

#Cambiar números a texto

In [82]:
# Function to convert numbers to words in Spanish
def convert_number_to_spanish(text):
    if not isinstance(text, str):
        return text  # Return original value if not a string

    # Split string into list of words
    temp_str = text.split()
    # Initialize an empty list for processed words
    new_string = []

    for word in temp_str:
        # If word is a digit, convert the digit to words in Spanish
        if word.isdigit():
            temp = num2words(int(word), lang='es')
            new_string.append(temp)
        else:
            # Append the word as it is
            new_string.append(word)

    # Join the words in new_string to form a single string
    return ' '.join(new_string)

# Apply the function to the 'steps' column
df['converted_steps'] = df['steps'].apply(convert_number_to_spanish)


In [83]:
df.head(1000)

,title,ingredients,steps,converted_steps
0,arepas de queso,1 taza de harina de arepa blanca o amarilla\r\...,"combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q..."
1,sudado de pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal..."
2,sancocho trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ..."
3,sudado de posta o muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","en una olla grande ponga la carne, el caldo, ...","en una olla grande ponga la carne, el caldo, e..."
4,sopa de lentejas colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...
7490,receta fácil y deliciosa,12 discos de empanada 500 gramos de osobuco 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...
7491,receta fácil y saludable,6 tapas de empanadas vegetarianas o veganas 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...
7492,canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 para empezar con la receta de las canastitas...,uno para empezar con la receta de las canastit...
7493,tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 para empezar con la receta de tacos de canas...,uno para empezar con la receta de tacos de can...


#Cambiar fracciones a texto

In [84]:
# Mapping of Unicode fractions to words
unicode_fraction_map = {
    '¼': 'un cuarto',
    '½': 'un medio',
    '¾': 'tres cuartos',
    '⅓': 'un tercio',
    '⅔': 'dos tercios',
    '⅕': 'un quinto',
    '⅖': 'dos quintos',
    '⅗': 'tres quintos',
    '⅘': 'cuatro quintos',
    '⅙': 'un sexto',
    '⅚': 'cinco sextos',
    '⅛': 'un octavo',
    '⅜': 'tres octavos',
    '⅝': 'cinco octavos',
    '⅞': 'siete octavos',
}

# Function to replace fractions in text
def convert_unicode_fractions(text):
    if not isinstance(text, str):
        return text  # Skip non-string inputs

    # Replace each Unicode fraction with its text equivalent
    for unicode_fraction, word in unicode_fraction_map.items():
        text = text.replace(unicode_fraction, word)

    return text


# Apply the function to the 'steps' column
df['converted_steps_2'] = df['converted_steps'].apply(convert_unicode_fractions)



In [85]:
df.head(1000)


,title,ingredients,steps,converted_steps,converted_steps_2
0,arepas de queso,1 taza de harina de arepa blanca o amarilla\r\...,"combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q..."
1,sudado de pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal..."
2,sancocho trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ..."
3,sudado de posta o muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","en una olla grande ponga la carne, el caldo, ...","en una olla grande ponga la carne, el caldo, e...","en una olla grande ponga la carne, el caldo, e..."
4,sopa de lentejas colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...,...
7490,receta fácil y deliciosa,12 discos de empanada 500 gramos de osobuco 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...,uno para empezar con la receta de las empanada...
7491,receta fácil y saludable,6 tapas de empanadas vegetarianas o veganas 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...,uno para empezar con la receta de las empanada...
7492,canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 para empezar con la receta de las canastitas...,uno para empezar con la receta de las canastit...,uno para empezar con la receta de las canastit...
7493,tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 para empezar con la receta de tacos de canas...,uno para empezar con la receta de tacos de can...,uno para empezar con la receta de tacos de can...


#Cambiar todo a español

In [86]:
# Function to detect the language of a given text
def detect_language(text):
    try:
        return detect(text)  # Detects the language and returns a language code (e.g., 'es' for Spanish)
    except:
        return None  # Return None if language detection fails

# Apply the language detection function to the 'steps' column
df['language'] = df['steps'].apply(detect_language)

# Check for rows where the language is not Spanish ('es')
non_spanish_rows = df[df['language'] != 'es']

# Display rows in non-Spanish languages
print(non_spanish_rows)



                                   title  \
23         sancocho de gallina o valluno   
8888            empadinhas de camarao...   
14876       cocktail de bacardi especial   
15038  batida de halls (batido de halls)   
27981                fresh broccoli soup   

                                             ingredients  \
23     3 ears fresh corn cut into 3 pieces\r\n12 cups...   
8888   para el recheio 1 kg de camarao limpo 1 cebola...   
14876  3/4 de ron bacardí 1/4 de zumo de limón gotas ...   
15038  1/2 garrafa de vodka 1 e 1/2 de halls de moran...   
27981  2 tbs butter 1 small onion cut in quarters 2 c...   

                                                   steps  \
23     in a large pot, place the chicken, corn, aliño...   
8888   1 tempere os camaroes, com limao, alho e pimen...   
14876  1 de juan valera troya (sitges). 2 batir en co...   
15038  1 bata tudo junto no liquidificador por 20 seg...   
27981  1 in a medium sauce pan over medium heat, saut...   

             

In [89]:
df = df[df['language'] == 'es']

In [90]:
# Count the number of rows where the language is not 'es' (not Spanish)
non_spanish_count = (df['language'] != 'es').sum()

# Print the result
print(f"Number of rows with a language different from Spanish: {non_spanish_count}")


Number of rows with a language different from Spanish: 0


In [91]:
df.head(1000)

,title,ingredients,steps,converted_steps,converted_steps_2,language
0,arepas de queso,1 taza de harina de arepa blanca o amarilla\r\...,"combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q...",es
1,sudado de pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal...",es
2,sancocho trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ...",es
3,sudado de posta o muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","en una olla grande ponga la carne, el caldo, ...","en una olla grande ponga la carne, el caldo, e...","en una olla grande ponga la carne, el caldo, e...",es
4,sopa de lentejas colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...,es
...,...,...,...,...,...,...
7491,receta fácil y saludable,6 tapas de empanadas vegetarianas o veganas 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...,uno para empezar con la receta de las empanada...,es
7492,canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 para empezar con la receta de las canastitas...,uno para empezar con la receta de las canastit...,uno para empezar con la receta de las canastit...,es
7493,tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 para empezar con la receta de tacos de canas...,uno para empezar con la receta de tacos de can...,uno para empezar con la receta de tacos de can...,es
7494,pionono de jamón y queso,6 huevos m 75 gramos de azúcar moreno (1/2 taz...,1 para realizar la receta de pionono de jamón ...,uno para realizar la receta de pionono de jamó...,uno para realizar la receta de pionono de jamó...,es


·Deshacerse de caracteres especiales

In [ ]:
import string

In [113]:
# Function to remove punctuation and special characters
def remove_punctuation(text):
    if not isinstance(text, str):
        return text  # Return original value if not a string
    return text.translate(str.maketrans('', '', string.punctuation))

# Use .loc to apply the function to the 'converted_steps_2' column
df.loc[:, 'converted_steps_2_clean'] = df['converted_steps_2'].apply(remove_punctuation)




In [114]:
# Iterate through each row in the DataFrame and add word counts to the new columns
df.loc[:, 'steps_word_count'] = df['converted_steps_2_clean'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
df.loc[:, 'ingredients_word_count'] = df['ingredients'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

# Now the DataFrame has new columns with the word counts for each row
# You can calculate the min, max, and average for these new columns

# For 'converted_steps_2'
steps_min = df['steps_word_count'].min()
steps_max = df['steps_word_count'].max()
steps_avg = df['steps_word_count'].mean()

# For 'ingredients'
ingredientes_min = df['ingredients_word_count'].min()
ingredientes_max = df['ingredients_word_count'].max()
ingredientes_avg = df['ingredients_word_count'].mean()

# Print the results
print(f"Converted Steps 2 - Min: {steps_min}, Max: {steps_max}, Average: {steps_avg:.2f}")
print(f"Ingredientes - Min: {ingredientes_min}, Max: {ingredientes_max}, Average: {ingredientes_avg:.2f}")




Converted Steps 2 - Min: 6, Max: 1812, Average: 200.28
Ingredientes - Min: 2, Max: 209, Average: 39.88


In [116]:
df.head(1000)

,title,ingredients,steps,converted_steps,converted_steps_2,language,steps_word_count,ingredients_word_count,converted_steps_2_clean
0,arepas de queso,1 taza de harina de arepa blanca o amarilla\r\...,"combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q...","combine la harina de maíz, agua caliente, el q...",es,92,28,combine la harina de maíz agua caliente el que...
1,sudado de pollo,8 muslos de pollo sin la piel\r\n1 cucharada d...,"en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal...","en una olla grande, caliente el aceite vegetal...",es,92,78,en una olla grande caliente el aceite vegetal ...
2,sancocho trifásico,1 taza de cebolla picada\r\n1 pimientón rojo f...,"coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ...","coloque la cebolla, el pimientón, el ajo y el ...",es,103,114,coloque la cebolla el pimientón el ajo y el co...
3,sudado de posta o muchacho,"1 posta o muchacho de 3 a 4 libras, cortada en...","en una olla grande ponga la carne, el caldo, ...","en una olla grande ponga la carne, el caldo, e...","en una olla grande ponga la carne, el caldo, e...",es,85,70,en una olla grande ponga la carne el caldo el ...
4,sopa de lentejas colombiana,1 cucharadita de aceite vegetal\r\n1 taza de c...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...,cocinar el chorizo en el aceite en una olla gr...,es,129,66,cocinar el chorizo en el aceite en una olla gr...
...,...,...,...,...,...,...,...,...,...
7491,receta fácil y saludable,6 tapas de empanadas vegetarianas o veganas 1 ...,1 para empezar con la receta de las empanadas ...,uno para empezar con la receta de las empanada...,uno para empezar con la receta de las empanada...,es,296,40,uno para empezar con la receta de las empanada...
7492,canastitas de espinaca,10 hojas de espinaca 1 cebolla ½ taza de leche...,1 para empezar con la receta de las canastitas...,uno para empezar con la receta de las canastit...,uno para empezar con la receta de las canastit...,es,384,46,uno para empezar con la receta de las canastit...
7493,tacos de canasta receta de la abuela,1 kilogramo de tortillas de maíz taqueras ½ ta...,1 para empezar con la receta de tacos de canas...,uno para empezar con la receta de tacos de can...,uno para empezar con la receta de tacos de can...,es,520,65,uno para empezar con la receta de tacos de can...
7494,pionono de jamón y queso,6 huevos m 75 gramos de azúcar moreno (1/2 taz...,1 para realizar la receta de pionono de jamón ...,uno para realizar la receta de pionono de jamó...,uno para realizar la receta de pionono de jamó...,es,288,32,uno para realizar la receta de pionono de jamó...


In [ ]:
## Clean recipies

In [ ]:
#Remove extra white spaces
df["directions"] = df["directions"].str.strip().replace(r'\s+', ' ', regex=True)
df["ingredients"] = df["ingredients"].str.strip().replace(r'\s+', ' ', regex=True)


In [ ]:
df

#Tokenization

In [ ]:
from transformers import GPT2Tokenizer

In [ ]:
df

In [ ]:
# Drop the directions_tokens and ingredients_tokens columns
#df.drop(columns=['directions_tokens', 'ingredients_tokens', 'directions_lenght','ingredients_lenght'], inplace=True)
df.drop(columns=['directions_length','ingredients_length'], inplace=True)



In [ ]:
# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Assuming df is your DataFrame with 'directions' and 'ingredients' columns
# Tokenize the 'directions' and 'ingredients' columns

df['directions_tokens'] = df['directions'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df['ingredients_tokens'] = df['ingredients'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# Check the result
print(df[['directions_tokens', 'ingredients_tokens']].head())


In [ ]:
df

In [ ]:
# Check if there are null values in 'directions_tokens' and 'ingredients_tokens'
null_values = df[['directions_tokens', 'ingredients_tokens']].isnull().sum()

# Print the result
print(null_values)

In [ ]:
# Check if there are any NaN values in the directions_tokens and ingredients_tokens columns
nan_check = df[['directions_tokens', 'ingredients_tokens']].isna().sum()

print(nan_check)


In [ ]:
# Check the first few tokenized samples
print(df['directions_tokens'].head())
print(df['ingredients_tokens'].head())

# Decode a sample tokenized row to see the tokens converted back to text
sample_row = df['directions_tokens'].iloc[0]
decoded_sample = tokenizer.decode(sample_row)
print(decoded_sample)


In [ ]:
# Check for empty token lists
empty_directions = df[df['directions_tokens'].apply(lambda x: len(x) == 0)]
empty_ingredients = df[df['ingredients_tokens'].apply(lambda x: len(x) == 0)]

print(f"Empty directions tokens: {len(empty_directions)}")
print(f"Empty ingredients tokens: {len(empty_ingredients)}")


In [ ]:
# Check the lengths of the tokenized sequences
df['directions_length'] = df['directions_tokens'].apply(len)
df['ingredients_length'] = df['ingredients_tokens'].apply(len)

# View summary statistics of token lengths
print(df[['directions_length', 'ingredients_length']].describe())


In [ ]:
# Manually set the padding token ID (typically, you can use an unused token ID)
tokenizer.pad_token = tokenizer.eos_token  # Use end-of-sequence token as pad token
tokenizer.pad_token_id = tokenizer.encode(tokenizer.pad_token)[0]  # Get token ID for pad_token

# Check if the pad_token_id is set correctly
print(f"Pad Token ID: {tokenizer.pad_token_id}")


In [ ]:
# Padding to max length
max_length = 512  # Define your desired maximum length

# Apply padding to both directions and ingredients tokens
df['directions_tokens'] = df['directions_tokens'].apply(lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length])
df['ingredients_tokens'] = df['ingredients_tokens'].apply(lambda x: x[:max_length] + [tokenizer.pad_token_id] * (max_length - len(x)) if len(x) < max_length else x[:max_length])

# Check if padding is working as expected
print(df[['directions_tokens', 'ingredients_tokens']].head())



In [ ]:
# Check the pad_token_id to ensure it's valid
print(f"Pad Token ID: {tokenizer.pad_token_id}")


In [ ]:
# Check for invalid token IDs (e.g., negative or zero values)
invalid_directions = df[df['directions_tokens'].apply(lambda x: any(i < 0 for i in x))]
invalid_ingredients = df[df['ingredients_tokens'].apply(lambda x: any(i < 0 for i in x))]

print(f"Invalid directions tokens: {len(invalid_directions)}")
print(f"Invalid ingredients tokens: {len(invalid_ingredients)}")


In [ ]:
# Check that all values are lists
non_list_directions = df[~df['directions_tokens'].apply(lambda x: isinstance(x, list))]
non_list_ingredients = df[~df['ingredients_tokens'].apply(lambda x: isinstance(x, list))]

print(f"Non-list directions tokens: {len(non_list_directions)}")
print(f"Non-list ingredients tokens: {len(non_list_ingredients)}")


In [ ]:
# Check for non-integer values in the token lists
invalid_directions_values = df[df['directions_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]
invalid_ingredients_values = df[df['ingredients_tokens'].apply(lambda x: any(not isinstance(i, int) for i in x))]

print(f"Invalid directions token values: {len(invalid_directions_values)}")
print(f"Invalid ingredients token values: {len(invalid_ingredients_values)}")


In [ ]:
# Check data types
print(df['directions_tokens'].apply(type).value_counts())
print(df['ingredients_tokens'].apply(type).value_counts())


In [ ]:
# Inspect the first few tokenized entries to see the output
sample_directions = df['directions'].head().apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
sample_ingredients = df['ingredients'].head().apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

print(f"Sample directions tokens: {sample_directions}")
print(f"Sample ingredients tokens: {sample_ingredients}")


In [ ]:
import torch

# Convert padded token lists to tensors
directions_tensor = torch.tensor(df['directions_tokens'].tolist())
ingredients_tensor = torch.tensor(df['ingredients_tokens'].tolist())

# Check the shape of the tensors
print(directions_tensor.shape)
print(ingredients_tensor.shape)


#Trainning Phase

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Combine into a dataset (X, Y)
dataset = TensorDataset(directions_tensor, ingredients_tensor)

# Create DataLoader for batching
batch_size = 8  # You can adjust this based on your system's capacity
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
